<a href="https://colab.research.google.com/github/leen204/mining-project/blob/main/DATA%20MINING%20NOTEBOOK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Goal


Goal: The goal of this project is to analyze the dataset, identify significant risk factors, and develop predictive models to assist in the early detection and understanding of Alzheimer’s Disease.


#Source


https://www.kaggle.com/datasets/rabieelkharoua/alzheimers-disease-dataset

#Dataset information


Number of attributes: 13

Class label: Diagnosis

1- Age: integer attribute showing the age of the patient ranging from 60-90

2- Gender: Shows the gender of the patient where 0 is a male and 1 is a female

3- Ethnicity: integer number attribute where each number corresponds to a certain ethnicity
-0 is Caucasian
-1 African American
-2 Asian
-3 Other

4- Education level: integer number attribute where each number corresponds to the education level of the patient
-0 None
-1 High School
-2 Bachelor's
-3 Higher

5- BMI: Continuous number attribute which stands for Body Mass Index of the patients, ranging from 15-40

6- Physical Activity: Continuous number atrribute that shows the Weekly physical activity in hours, ranging from 0-10

7- Sleep Quality: Continuous number attribute that shows Sleep quality score, ranging from 4-10

8- Family History: number attribute that acts as a boolean determining whether the patient's family has a history of alzheimer or not

9- Cholesterol total: continuous number attribute showing total cholesterol levels, ranging from 150-300 mg\Dl

10- MMSE: continuous number attribute which stands for Mini-Mental State Examination score, ranging from 0 to 30. Lower scores indicate cognitive impairment.

11- Forgetfulness: number attribute that acts as a boolean to specify whether the patiens has struggled with forgetfulness or not

12- Diagnosis: the class label of our dataset where 0 indicates that the patient doesn't have an alzheimer diagnosis and 1 indicates otherwise

